In [1]:
import glob
import os

import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [10]:
from PIL import Image


def generate_image_from_video(video_file, output_dir):
    vidcap = cv.VideoCapture(video_file)
    success = True
    count = 0
    success, image = vidcap.read()
    while success:
        image = image[:, :, ::-1]
        # print(image.shape)
        # img_obj = Image.fromarray(image).resize((608, 608))
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
        # image.save(os.path.join(output_dir, f"frame_{count:06}.jpg"))
        cv.imwrite(os.path.join(output_dir, f"frame_{count:06}.jpg"), image)
        count += 1
        success, image = vidcap.read()

video_file = "Shrimp/sick_shrimp.mp4"
output_dir = "Shrimp/data/obj_train_data/"
generate_image_from_video(video_file, output_dir)

(480, 848, 3)


In [ ]:
def get_corrected_bounding_box(annot_idx, annot_dir, img_dim):
    annot_file = f"{annot_dir}/frame_{annot_idx:06}.txt"
    with open(annot_file) as infile:
        _, x_center, y_center, annot_width, annot_height = [
            float(elem) for elem in infile.readline().strip("\n").split()
        ]

    img_height, img_width = img_dim

    x1 = int(img_width * (x_center - annot_width / 2))
    y1 = int(img_height * (y_center - annot_height / 2))

    norm_annot_width = int(annot_width * img_width)
    norm_annot_height = int(annot_height * img_height)
    
    return (x1, y1, norm_annot_width, norm_annot_height)


def play_video(video_input, annotation_dir):
    cap = cv.VideoCapture(video_input)
    idx = 0

    cv.startWindowThread()
    while cap.isOpened():
        ret, frame = cap.read()
        # if frame is read correctly ret is True
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        gray = cv.cvtColor(frame, cv.COLOR_RGB2GRAY)

        x1, y1, w, h = get_corrected_bounding_box(idx, annotation_dir, gray.shape)

        cv.rectangle(gray, (x1, y1), (x1+w, y1+h), (255, 0, 0), 2)
        cv.imshow("frame", gray)
        idx += 1
        if cv.waitKey(1) == ord("q"):
            break
        # if cv.getWindowProperty('frame', cv.WND_PROP_VISIBLE) < 1:
        #     break
    cap.release()
    cv.destroyAllWindows()
    for i in range(10):
        cv.waitKey(1)


video_file = "Shrimp/sick_shrimp.mp4"
annotation = "Shrimp/sick_shrimp_track/obj_train_data/"
play_video(video_file, annotation)

In [ ]:
# import matplotlib.patches as patches

# frame_count = 6000

# for i in range(frame_count):
#     if i < 5950 or i > 5980:
#         continue

#     img_file = f"Shrimp/data/obj_train_data/frame_{i:06}.PNG"
#     annot_file = f"Shrimp/sick_shrimp_track/obj_train_data/frame_{i:06}.txt"
#     with open(annot_file) as infile:
#         _, x_center, y_center, annot_width, annot_height = [
#             float(elem) for elem in infile.readline().strip("\n").split()
#         ]

#     img = cv.imread(img_file)
#     img_height, img_width, _ = img.shape

#     x1 = img_width * (x_center - annot_width / 2)
#     y1 = img_height * (y_center - annot_height / 2)

#     norm_annot_width = annot_width * img_width
#     norm_annot_height = annot_height * img_height

#     # Create figure and axes
#     fig, ax = plt.subplots()
#     # Display the image
#     ax.imshow(img)
#     # Create a Rectangle patch
#     rect = patches.Rectangle(
#         (x1, y1),
#         norm_annot_width,
#         norm_annot_height,
#         linewidth=1,
#         edgecolor="r",
#         facecolor="none",
#     )
#     # Add the patch to the Axes
#     ax.add_patch(rect)
#     ax.set_title(f"{i}")

#     plt.show()

In [ ]:
# Load a model
model = YOLO("yolov8n.yaml").load(custom_model)

# Run batched inference on a list of images
results = model(['im1.jpg', 'im2.jpg'])  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs